# Rapport de TPs - Apprentissage Supervisé (KNN-ANN-SVM)

## Auteur : Anaïs RABARY (5SDBD - Gr A)
### Date : 7 Décembre 2018


NOTES : Le beau rapport avec un plan et des images dimensionnées ainsi que les codes des TP 1, 2 et 3 sont disponibles sur ce git : https://github.com/anaisrabary/Tps_ApprentissageSupervise .

## Introduction
<a id="intro"></a>
Dans le cadre des Travaux Pratiques sur l'Apprentissage Supervisé réalisés en 5ème Année SDBD (Système Dsitribués Big Data), sous la supervision de M-V. Le Lann et M. Siala, nous avons étudiés différentes méthodes d'apprentissage supervisé : KNN, K-Neighrest Neighbors ou les algo des K plus proches voisins, ANN, Artificial Neural Network ou Réseau Artificiel de Neurones et SVM, Support Vector Machine ou machine à support de vecteur. L'étude est réalisée à partir de l'outil [Scikit-Learn sur python](https://scikit-learn.org/).

L'objectif de ce rapport est de présenter ces 3 méthodes puis de les comparer. Ce rapport se veut pédagogique pour former un support personnel qui pourra être réutilisé pour des études futures. On présentera notamment les paramètres entrant dans l'étude de comparaison. Les codes exécutés pour réaliser l'étude seront rendus dans leur totalité dans les autres notebook. Je n'ai pas jugé nécessaire de surcharger ce rapport et d'entraver sa bonne lisibilité.

Ces 3 méthodes ont pour but de classifier les données. Afin de pouvoir les évaluer, nous travaillerons sur un même jeu de données, [MNIST](http://yann.lecun.com/exdb/mnist/), base de données ouvertes de chiffres écrits à la main. Dans ce cas, le but sera d'arriver à déterminer les 10 classes de chiffres. 
Les éléments de comparaison dépendront des méthodes, mais nous analyserons à chaque fois leurs performances et leur temps d'éxécution. On s'attardera aussi sur les choix des paramètres permettant d'obtenir les meilleurs résultats pour la classification sur le jeu de données mnist.


## PLAN


[Introduction](#introduction)  
1. [Présentations générales](#markdown-header-présentations-générales)  
    1.1 [Le Dataset](#dataset)  
    1.2 [KNN](#KNN)  
    1.3 [ANN](#ANN)   
    1.4 [SVM](#SVM)  
2. [Etudes séparées](#etud)  
   2.1 [Méthodes de vérification](#verif)  
   2.2 [Indicateurs de performance](#perf)  
   2.3 [Choix des paramètres de l'algorithme](#choix)  
      2.3.1 [Mnist et KNN](#paramKNN)  
      2.3.2 [Mnist et les neuronnes](#paramANN)  
      2.3.3 [Mnist et SVM](#paramSVM)  
3. [Analyse comparative](#comp)  
[Conclusion](#conclusion)  



## 1. Présentations générales
<a id="presgen"></a>

### 1.1 Le Dataset
<a id="dataset"></a>

Le dataset utilisé pour la comparaison des algorithmes, les chiffres de 0 à 9 écrits à la main, contient 70 000 images labélisées. Ces images font 28x28 pixels, et donc comportent 784 pixels. 
L'objectif des algorithmes développés ici est de pouvoir classifier ces images et ainsi pouvoir prédire le chiffre d'une image non labélisée. 
Ci-après voici un exemple d'affichage des données mnist.

<div class="row" style="margin-top: 10px">
    <div class="col-md-5">
        <img src="images_rapport/MnistExamples.png" style=" width: 300px;" />
    </div>
    <div class="col-md-5">
        <img src="images_rapport/mnist_example.png" style=" width: 200px;" />
    </div>
</div>

Ce jeu de données a un but pédagogique. Les 70000 images sont donc toutes labélisées. On veillera donc à séparer notre jeu en 2 parties : une partie pour l'apprentissage et une partie pour le test. Grâce aux labels des données de tests, on pourra mesurer le score de notre modèle. Le jeu comprends plusieurs attributs dont notamment data et target qui vont nous être utiles pour apprendre, tester et vérifier nos models:
- data, un tableau de n instances x m attributs
- target, le label associé à chaque instance.

### 1.2 KNN
<a id="KNN"></a>

On rappel ici les principes de KNN :
L'algorithme des k-plus proches voisins, ou K Neighrest Neighbors, est un algorithme simple et intuitif. Il est utilisé pour des problèmes de classification.
Pour chaque objet $o$ que l'on souhaite classifier, on cherche ses $K$ plus proches voisins connus (labélisés). Puis on associe à $o$ le label qui est majoritaire parmis ses voisins. 

On effectue les tests en utilisant le model de scikit-learn, sklearn.neighbors, qui est paramétrable.

Pour trouver les K plus proche voisins, l'algorithme KNN repose sur une mesure de distance. Cependant, il existe plusieurs types de distances différentes :
- Euclidienne : $\sqrt{\sum (x-y)^2}$
- Manhattan : $\sum|x-y|$
- Chebyshev :$max(|x-y|)$
- Minkowski :$(\sum|x-y|^p)^{1/p}$
- Hamming : $\sum|x-y|$ avec $x=y \implies D=0$ et $x\neq y \implies D=1$

Par défaut, c'est la distance euclidienne qui est utilisée. Une étude comparative de ces distances est réalisée dans la partie 2.3.1.

Voici ci-après le squelette de code qui servira à l'étude des performances de KNN sur Mnist. On rappelle que la totalité du code ayant servie à l'étude de KNN est donnée dans le notebook dédié.



In [ ]:
#Imports liés à KNN et Mnist
from sklearn import datasets
import numpy as np
from sklearn.model_selection import train_test_split 
from sklearn import neighbors
#Imports pour la mesure de performance
import matplotlib.pyplot as plt
from sklearn import metrics
import time

# Charger les données
mnist=datasets.fetch_mldata('MNIST original')

#ECHANTILLONS DE 10000
indices = np.random.randint(70000, size=10000)
data = mnist.data[indices]
target = mnist.target[indices]
#pour séparer le dataset en 2 échantillons de training et de test
#Data fait donc 8000 et test 2000
xtrain, xtest, ytrain, ytest =train_test_split(data, target, train_size=0.8)

# Classifier KNN, apprentisage, prédiction et score
clf=neighbors.KNeighborsClassifier(10)
clf.fit(xtrain,ytrain)
predict = clf.predict(xtest)
score = clf.score(xtest,ytest)


### 1.3 ANN
<a id="ANN"></a>
On présente ici un réseau de neurones multicouches, perceptron multi-couche (MLP, Multi-layered percepron ou Artificial Neural Network). On peut distinguer 3 parties :
- les entrées : permettent de prendre un vecteur à plusieurs dimensions (une dimension par neurone),
- les sorties : présentent le résultat,
- les couches cachées (hidden layers) : permettes de faire une séparation entre les données, par combinaison linéaire.

Sur les liens entre chaques neurones se trouvent des **poids** $w_0,w_1,w_2$ (weight). C'est ces poids qui sont adaptés pendant l'apprentissage. Dans un neurone, on cacule la somme pondérée de tous les signaux entrants, $\sum(w_i\dot x_i)$. Puis on y ajoute un **biais** $b$, qui permet de décaller la fonction d'activation. En sortant du neurone, le signal calculé passe par une **fonction d'activation** qui laisse passer un signal suivant un seuil. On a enfin le signal de sortie du neuronne.

<div class="row" style="margin-top: 10px">
    <div class="col-md-5">
        <img src="images_rapport/MLP/MLPSchema.jpg" style=" width: 300px;" />
    </div>
    <div class="col-md-5">
        <img src="images_rapport/MLP/neurone_detail.png" style="width: 250px;" />
    </div>
</div>


Lors de la création d'un tel réseau, les poids et biais sont initialisés aléatoirement. Puis c'est par une méthode de backpropagation qu'ils sont recalculés jusqu'à la fin de l'apprentissage.

Le modèle proposé par scikit learn est paramétrable.  
Pour les neurones dans les couhces cachés, il y a plusieurs fonction d'activation possibles :
- identity, $f(x)=x$ donc pas de fonction d'activation,
- logistic, $f(x)=1/(1+\exp (-x))$, la fonction d'activation du sigmoid,
- tanh, $f(x)=tanh(x)$, la fonction hyperboliques de tan,
- relu, $f(x)=max(0,x)$, la fonction de rectification linéaire unitaire.
C'est relu qui est utilisé par défaut.  

On peut aussi modifier le solver utilisé pour calculer les poids :
- lbfgs, un optimiseur de la famille des méthodes quasi Newtonienne,
- sgd, la méthode de déscente de gradient stochastic,
- adam, une autre méthode de descente de gradient, optimisée par Kingma, Diederik et Jimmmy Ba.
Par défaut, c'est adam qui est utilisé.

Il est aussi possible de paramétrer alpha $\alpha$, qui est le paramètre de régularisation dans la pénalité L2. Il permet de prévenir l'overfitting, en contraignant la taille des poids. 

Choisir une architecture de réseau de neurone demande une certaine expertise. Il faut essayer différentes architectures et choisir le réseau donnant le meilleur résultat de généralisation sur les tests.



Voici ci-après le squelette de code qui servira à l'étude des performances de ANN sur Mnist. On rappelle que la totalité du code ayant servie à l'étude de ANN est donnée dans le notebook dédié.


In [ ]:
#import necessaires
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split 
from sklearn.neural_network import MLPClassifier
from sklearn import metrics
import time

#charger le jeu de données MNIST
mnist=datasets.fetch_mldata('MNIST original')

#randomise Data et target
indices = np.random.randint(70000, size=70000)
data = mnist.data[indices]
target = mnist.target[indices]
# on veut un training set de 49000
xtrain, xtest, ytrain, ytest =train_test_split(data, target, train_size=49000)

clf = MLPClassifier(hidden_layer_sizes=(50))
clf.fit(xtrain, ytrain)
predict = clf.predict(xtest)
score = clf.score(xtest,ytest)

recall = metrics.recall_score(ytest, predict, average ='macro')
precision = metrics.precision_score(ytest, predict,  average='macro')
loss01 = metrics.zero_one_loss(ytest, predict)


print("Ce modèle de MLP, d'1 couche de 50, à un score de ", score*100, "%.")
print("4eme image : prédiction ",predict[3], "reel : ", ytest[3])
print ("ce modèle de MLP à une précision de", precision*100, "%.")
print ("ce modèle de MLP à un recall de",recall*100, "%.")
print ("ce modèle de MLP à un zero-one_loss de",recall*100, "%.")
print("    temps apprentissage : ", timetrain, "sec , temps prediction = ", timePred, "sec." )


#Ce modèle de MLP, d'1 couche de 50, à un score de  98.5047619047619 %.
#4eme image : prédiction  5.0 reel :  5.0
#ce modèle de MLP à une précision de 98.50503688809177 %.
#ce modèle de MLP à un recall de 98.49104431028593 %.
#ce modèle de MLP à un zero-one_loss de 98.49104431028593 %.
#    temps apprentissage :  322.9891335964203 sec , temps prediction =  0.6702065467834473 sec.

### 1.4 SVM
<a id="SVM"></a>

La méthode d'apprentissage par Machine à Vecteurs de Support ou SVM est la dernière méthode que nous allons voir dans ce rapport. Le but de SVM est de maximiser la marge de séparation entre les classes. Cela permet en fait de séparer linéairement nos données en augmentant la dimension. La méthode permet de trouver un hyperplan qui sépare nos données, tout en maximisant les marges.

<div class="row">
     <img src="images_rapport/SVM/SVM_2.png" style=" width: 300px;" />
</div>

Nous allons utiliser SVC de Scikit-learn.

Il est possible de faire varier plusieurs paramètres. 
**Le kernel** :
- linéaire
- poly
- rbf
- sigmoid
Par défaut c'es RBF qui est utilisé
 
Il est aussi possible de faire varier **le coût** et **gamma**.

Voici ci-après le squelette de code qui servira à l'étude des performances de SVM sur Mnist. On rappelle que la totalité du code ayant servie à l'étude de SVM est donnée dans le notebook dédié.

In [ ]:
#import necessaires
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn import metrics
import matplotlib.pyplot as plt
import numpy as np
import time

#charger le jeu de données MNIST
mnist=datasets.fetch_mldata('MNIST original')

#randomise Data et target
indices = np.random.randint(10000, size=10000)
data = mnist.data[indices]
target = mnist.target[indices]
#pour séparer le dataset en 2 échantillons de trainint et de test
# on veut un training set de 49000 (70% training set - 30 % test set)
xtrain, xtest, ytrain, ytest =train_test_split(data, target, train_size=int(len(data)*0.7))

clf = SVC(kernel='rbf')
startTrain =time.time()
clf.fit(xtrain, ytrain)
endTrain = time.time()
# PREDIC
startpred= time.time()
predict = clf.predict(xtest)
endpred = time.time()
# METRICS
score = clf.score(xtest,ytest)
recall = metrics.recall_score(ytest, predict, average ='macro')
precision = metrics.precision_score(ytest, predict,  average='macro')
loss01 = metrics.zero_one_loss(ytest, predict)
timetrain = endTrain - startTrain
timePred = endpred - startpred

print("Ce modèle SVC avec un kernel", ker, "a un score de ", score*100, "%.")
print("4eme image : prédiction ",predict[3], "reel : ", ytest[3])
print ("précision :", precision*100)
print ("recall  :",recall*100)
print ("zero-one_loss :",recall*100)
print( "training time :", timetrain)
print( "prediction time :", timePred)
    

## 2. Etudes séparées
<a id="etud"></a>

### 2.1 Méthode de vérification
<a id="verif"></a>

Il existe 3 grandes méthodes de vérifications : 
-	Test set validation (holdout method)
-	K-fold cross-validation
-	Leave-one-out cross-validation


La 1ère, Test Set validation est la plus simple. Elle consiste à séparer le jeu de données en 2. Une partie est réservée pour l'apprentissage, l'autre pour le test et donc la validation du modèle. Cela peut être par exemple, réserver 70% des données pour l'apprentissage et les 30% restants pour les tests.

Cependant, si l'on changeait nos de tests, mais qu'on gardait le même modèle, le score serait surement différent. De même, en choisissant un modèle dont l'apprentissage repose sur les même données, somme nous certain que l'apprentissage aurait été le même pour d'autres données ? Il est vrai que toutes les données servant à tester ne font pas parties des données d'apprentissage.

Voilà tout l'intérêt des 2 prochaines méthodes qui reposent sur la même idée. Le but est d'apprendre et tester notre modèle sur toutes les données. Dans le cas du K-fold cross validation, on divise le jeu de données en K parties égales. Puis on défini notre test set comme étant l'une des $K$ parties. Les autres $K-1$ parties sont regroupées pour former le training set. On fait apprendre notre classifier. Puis on change de test set en prenant la partie suivante et on réitère $K$ fois jusqu'à ce que toutes les parties aient été un test set. 

 <div class="raw">
        <img src="images_rapport/K-fold_cross_validation.jpg" style="width: 300px;" />
 </div>  
 
Il est recommandé de faire un 10-fold cross validation.
La méthode du leave-one out cross validation correspond à la méthode du K-fold cross validation, poussé à l'extrème, pour $k=n$. En effet, à chaque itération, le test set est composé d'une seule data et le training set est composé de $n-1$ data.

### 2.2 les indicateurs de performance
<a id="perf"></a>

On peut utiliser différents indicateures pour mesurer la performance des algorithmes.   

**Le score** nous donne tout simplement le pourcentage de réussite de classification des données de test. Il représente en fait la moyenne exacte des data prédites $xtest$ en fonction de leur label $ytest$.  

**Le recal ou sensibilité** renvoie le ratio $Tp/(Tp+Fn)$ où $Tp$, true prositive et $Fn$, false negative. Il représente l'abilité du classifier de trouver les exemples positifs. C'est la probabilité de détection.  

**La précision** retourne le ratio $Tp/(Tp+Fp)$ où $Fp$ false positive. Cet indicateur présente la capacité d'un classifier à ne pas labéliser positivement un label qui est négatif.  

**L'erreur zero-to-one loss**, retourne le ratio d'examples mal classifiés. La meilleur performance. Il me semble que le zero-to-one loss est en fait $1-score$.  

**La mean squared error**, $ MSE(y_{pred},y_{label})=\frac{1}{n_{samples} }\sum (y_{labeli} - y_{predi})^2 $.  

**Les temps de training et de prediction**.

Note : pour le recall et la précision, il faudra setter le paramètre average à macro et non micro.

Voici ci-après un exemple de code avec les métriques utilisées, sur un exemple de KNN :

In [ ]:
#Classifier
clf=neighbors.KNeighborsClassifier(k)
# temps de training et prédiction mesuré
startTrain =time.time()
clf.fit(xtrain,ytrain)    
endTrain = time.time()
startpred= time.time()
clf.predict(xtest)
endpred = time.time()
 # Metrics (score, precision, recall, zero-to-one loss, temps de training et de predicion)
predict = clf.predict(xtest)
score = clf.score(xtest,ytest)
precision =  metrics.precision_score(ytest, predict,  average='macro')
recall = metrics.recall_score(ytest, predict, average ='macro')
loss01 = metrics.zero_one_loss(ytest, predict)
timetrain = endTrain - startTrain
timePred = endpred - startpred


### 2.2 Choix des paramètres de l'algorithme
<a id="choix"></a>

#### 2.2.1 Mnist et KNN
<a id="paramKNN"></a>

Nous avons réalisé plusieurs études sur KNN et les données de Mnist. Le training set était composé de 8000 données et le test set de 2000 données

Tout d'abord, on a cherché à **déterminer le meilleur K** pour nos données MNIST. On a donc entrainé et testé notre KNN pour des K différents (de 2 à 15).

 <div class="raw">
        <img src="images_rapport/KNN/KNN_samedata_Kchange.png" style="width: 400px;" />
 </div>  

On peut constater sur le graphe ci-dessus, que le meilleur score est atteint pour K=3 (94,8%) avec une bonne précision (94,83%) ainsi que le meilleur recall (94,62%). On notera que la précision pour K=5 est légèrement meilleur(94,90%).  

Les **temps de training sont faibles**(entre 0,4 et 0,7sec). Ceux de **prédiction sont plus élevés** mais globallement compris entre 22,5 et 25 sec. Les variations observées ici ne sont pas conséquentes et dépendent des autres processus s'éxécutant sur l'ordinateur. Dans l'étude ci-dessous, le temps de prédiction est plus faible. Je pense que c'est dû à mon ordinateur car j'ai mené cette étude un jour différent.  

 <div class="raw">
        <img src="images_rapport/KNN/KNN_K3_15fold.png " style="width: 400px;" />
 </div>

En faisant un **15-fold validation pour K=3**, on se rend compte que le modèle est plus sensible suivant les données d'entrainement. Mais cela reste correct car le score est compris entre 94% et 96%. En aucun cas le K-fold révèle un score en dessous de 94%.  

Si l'on regarde l'effet de la taille du training et du test set sur le modèle pour k=3 on remarque que **plus le training set est grand, plus le score est bon**. Pour training set 10% le score est de 91%. Puis il augmente régulièrement jusque 96,5% pour un training set de 90%. De même la précision et le recall augmentent.  

D'autre part, si l'on fait varier le nombre de données utilisées pour entrainer et tester le modèle, on constate que plus le training set est grand, plus le score est meilleur. Pour un training set de 4000 données, le score est de 92,8%. Pour un training set de 20000 données, le score est de 96,4%. 

Cependant, il faut aussi penser au temps de prédiction qui est plus important avec l'augmentation du nombre de données. C'est pour cela que l'algorithme KNN n'est pas adapté à de trop grosses quantités de données. En effet, KNN compare toutes les données de tests avec leur voisins (dans le training set), puis finalise le label. **Le temps de prédiction peut donc être très important si le training set est grand.**

KNN pour K=3 semble être un bon choix de paramètre. C'est un juste milieu entre le **underfitting et le overfitting**. Dans le premier cas, si K est trop petit, on risque de mal déterminer les classe. Dans le 2eme cas, on rique de "sur déterminer" les classes et de trop ressérer les frontières autour d'elles. Cela peut causer des mauvaises prédictions. Il faut donc trouver le juste milieu pour que le modèle puisse bien généraliser. 

Mais qu'en est-il des distances présentées dans la partie 1 ?

Voici les scores obtenus pour les différentes distances :
- distance Eucliedienne, le score =  96.45 %
- distance de Manhattan, le score =  93.65 %
- distance de Chebyshev, le score =  68.45 %
- distance de Hamming, le score =  74.0 %
- distance de Minkowski p=3 , le score =  95.3 %
- distance de Minkowski p=4, le score =  95.5 %
- distance de Minkowski p=5, le score =  95.85 %

On constate que dans le cas de mnist, la **distance euclidienne**, celle par défaut, nous donne le meilleur score. Il faut toutefois rester attentif à ce paramètre dans un autre domaine d'application.

Le dernier paramètre étudié est celui des jobs. Il correspond au nombre de jobs exécutés en parallèle pour faire tourner l'algorithme KNN. -1 signifie que l'algo utilise tous les processeurs disponibles. L'utilisation de **toutes les ressources** disponible réduit le temps de prédiction : 
- temps d'entrainement 1 job :  0.49367713928222656 sec.
- temps de prediction 1 job :  25.477837085723877 sec.
- temps d'entrainement -1 job :  0.5066468715667725 sec.
- temps d'prediction -1 job :  7.67464017868042 sec.


#### 2.2.2 Mnist et les neurones
<a id="paramANN"></a>

Pour pouvoir évaluer le meilleur réseau de neurones, il faut tester différentes architectures. Premièrement, on etudie **l'effet du nombre de couches cachées**. Pour cela, on fixe le nombre de neurones sur chaque couche à 50. Et on fait varier notre réseau de 1 à 100 couches cachées.   
 <div class="raw">
        <img src="images_rapport/MLP/changingHiddelayers.png" style="width: 400px;" />
 </div>


On remarque que passer 50 hidden layers, le réseau n'arrive même plus à classifier. IL a un score stagnant autour de 11%. Mais pour un réseau ayant entre 2 et 35 couches cachées, le score varie entre 96 et 98%

Ensuite on a essayer d'observer **l'effet du nombre de neurones sur chaque couche**. pour cela, on a créé 5 classifier comprenant entre 1 et 10 couches cachées et avec entre 10 et 300 neurones sur chaque couches :
- Reseau 1 : 1 couche avec 300 neurones
- Réseau 2 : 3 couches avec respectivement 20, 200 et 50 neurones
- Réseau 3 : 5 couches avec 50, 100, 200, 100, 50 neurones (allure gaussienne)
- Réseau 4 : 7 couches avec 300, 250, 200, 150, 100, 50, 10 neurones (nombres décroissants)
- Réseau 5 : 9 couches avec 30, 60, 90,120, 150, 180, 210, 240, 270 neurones (nombres croissants)

Ensuite, avec ces caractéristiques architecturales, on essaie différentes **méthodes de calcul des poids** (LBFGS, SGD et ADAM, comme vue dans la partie 1).
Dans le graphe ci-après, les points de gauche à droites peuvent être regrouper par 3. Chaque groupe de 3 représente un réseau avec dans l'ordre des solvers LBFGS puis SGD puis Adam.

 <div class="raw">
        <img src="images_rapport/MLP/weightmethod.png" style="width: 400px;" />
 </div>

Pour les réseaux 1, 3 et 5, les 3 méthodes de calcul des poids donnent un résultats plus que correct (entre 96 et 98%). Le réseau 4 nous révèle ses faiblesses. On peut l'éliminer de notre étude. Le réseau 2 est un modèle acceptable uniquement si ADAM est employé.
De façon générale, on remarque que la méthode LBFGS est plus lente à apprendre, avec un temps d'entrainement pouvant dépasser les 10 min pour 49000 données. C'est la méthode ADAM qui a le temps d'apprentissage le plus faible, pour les 5 réseaux testés.

On peut aussi comparer ces 5 réseaux en changeant leur **fonction d'activation**. On étudie ici les fonctions d'activation suivantes : identity, logistic, tanh et relu (cf partie 1 pour une présentation théorique). De gauche à droite, chaque groupe de 4 points représentes un réseau. 
 <div class="raw">
        <img src="images_rapport/MLP/activationfunction.png" style="width: 400px;" />
 </div>

Ici c'est le réseau 5 qui n'a pas réussi à classifier, avec la fonction d'activation logistique. On remarque globalement que c'est la focntion d'activation Relu, celle par défaut qui donnele meilleur score, la meilleur précision et le meilleur recall. Pour les temps d'apprentissage, c'est plus mitigé et cela dépend des réseaux. Avec la fonction d'activation logistique, le temps d'apprentissage est souvent plus élevé. Et avec Relu il est souvent plus faible.

On peut aussi faire varier le alpha qui sert à évitter l'overfitting. Un alpha faible encourragera des poids et un biais élevés, résultant en des frontières de classification plus lisses. Tandis qu'un alpha élevé encouragera des poids plus petits, résultat en des frontières potentiellement plus compliquées.

 <div class="raw">
        <img src="images_rapport/MLP/changingAlpha.png" style="width: 400px;" />
 </div>


Sur nos 5 réseaux, on constate que un alpha trop élevé (1000) ne permet plus de classifier nos données. Le score chute à 11%. En regardant les temps d'apprentissage, un faible alpha (0,001 ou 0,1) est préférable.



Avec l'étude sur ces 5 réseaux, on remaque que le réseau 4 qui a un nombre croissant de neurones sur chaque couche (forme de pyramide) n'est pas idéal. Au contraire avoir un pyramide inversée, comme avec le réseau 5, ou une forme losange comme le **réseau 3**, offre un bon score de classification. Pour les méthodes de poids, il faut privilégier **ADAM** qui offre le meilleur score pour le temps d'apprentissage le plus faible. De même, la fonction d'activation par défaut, **relu**, semble être la fonction la plus adéquate. Enfin, il faut privilégier un **alpha entre 0,001 et 0,1** pour ne pas avoir de hunderfitting et ne plus pouvoir classifier nos données.



#### 2.3.3 Mnist et SVM
<a id="paramSVM"></a>

Pour etudier le modèle SVC de scikitlearn, sur mnist, on commence par étudier les différents **kernels**.

<div class="raw">
        <img src="images_rapport/SVM/kernel.png" style="width: 400px;" />
 </div>
 On constate ici que les kernel linéaire et poly ont un score satisfaisant (98,6% et 99,5%), comparé aux kernel rbf et sigmoid. De même, leur temps d'apprentisage et de prédiction sont très faibles (environ 11 et 5 sec).
 
 Maintenant, on fait varier le paramère **C**, avec un kernel poly :
 <div class="raw">
        <img src="images_rapport/SVM/changeC.png" style="width: 400px;" />
 </div>
 
Pour C, rien ne change, le score reste à 99,56%, seul le temps d'entrainement et de prédiction varient. C'est à cause des autres processus s'exécutant sur mon ordinateur.

En faisant varier **gamma**, on a les même constats. Le graphe n'est donc pas rapporté ici, pour ne pas surcharger le rapport.

Sikit-learn nous offre un outils : GridSearchCV, qui va trouver la meilleur combinaison suivant les paramètres qu'on lui donne. (Le seul inconvénient de cet outil est qu'il est long à exécuter). Voici donc comment l'exécuter :

In [ ]:
# PARTIE AVEC GridSearchCV

paramGrid = {'C': [0.001, 0.01, 0.1, 1, 10, 100],
              'gamma': [0.0001, 0.001, 0.01, 0.1],
              'kernel': ('linear', 'poly')
             }

grid = GridSearchCV(SVC(), paramGrid, cv=5)
grid = grid.fit(X=xtrain, y=ytrain)
print (grid.best_params_)

#The best param are : {'C': 0.001, 'gamma': 0.0001, 'kernel': 'linear'} for 10000 data

Avec les meilleurs paramètres choisi pour 10000 données, on le fait tourner en augmentant les données, jusqu'à 50000. On se rend compte que le modèle ne généralise pas bien. Il faudrait refaire une études pour changer les paramètres pour des données plus importantes. Cependant, cela demande du temps.

 <div class="raw">
        <img src="images_rapport/SVM/augmentData.png" style="width: 400px;" />
 </div>

## 3. Analyse comparative
<a id="comp"></a>
Pour pouvoir comparer les 3 méthodes rappelons les performances obtenues :
- pour KNN : pour 20000 données, 96,5% pour un temps de training négligeable et un temps de prédiction de 30sec
- pour ANN : pour 70000 données 98,5%, un temps d'entrainement de 5min et un temps de prédiction négligeable
- pour SVM : pour 20000 données, un score de 98,78% pour un temps de training de 10 sec et un temps de prédiction de 5 sec.

A première vu, SVM est donc le meilleur modèle en fonction du score et des temps d'entrainement et de training. Mais on a aussi vu que SVM ne généralisait pas bien car si l'on garde le même modèle, on tombe à 96% pour des temps d'entrainement et de prédiction de 4min.

On remarque aussi qu'entre KNN et MLP, le temps d'apprentissage et de prediction ont été inversé. En effet, KNN apprend très rapidement, mais il doit tester tous les voisins avant de pouvoir labéliser les données de test donc son temps de prédiction  est long. Inversement, le Réseau de neuronne doit stabiliser ses poids à l'entrainement puis la prédiction est instantannée.

En ce sens, ANN est à préconnisé si on l'adapte pour des données reçues en streaming, ou si l'on doit prédire un grand nombre de données. Au contraire, on ne recommandera pas KNN qui est trop coûteux en temps de prédiction.

Globalement, les 3 modèles permettent bien de classifier nos données mnist.

Personnellement, je suis plus à l'aise avec les modèles de KNN et ANN. J'ai du mal à voir les effets de C et de gamma dans le modèle de SVM.

In [ ]:
#Meilleur KNN
# K=3
# Toutes les ressources (njobs=-1)
# distance euclidienne par défaut

# imports mnist et KNN
from sklearn.datasets import fetch_mldata
from sklearn import datasets
from sklearn.model_selection import train_test_split 
from sklearn import neighbors
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics
import time

mnist=datasets.fetch_mldata('MNIST original')


#ECHANTILLONS DE 10000
indices = np.random.randint(70000, size=20000)
data = mnist.data[indices]
target = mnist.target[indices]
#pour séparer le dataset en 2 échantillons de trainint et de test
xtrain, xtest, ytrain, ytest =train_test_split(data, target, train_size=0.8)


#Classifier
clf=neighbors.KNeighborsClassifier(3,n_jobs=-1)
# temps de training et prédiction mesuré
startTrain =time.time()
clf.fit(xtrain,ytrain)    
endTrain = time.time()
startpred= time.time()
clf.predict(xtest)
endpred = time.time()
 # Metrics (score, precision, recall, zero-to-one loss, temps de training et de predicion)
predict = clf.predict(xtest)
score = clf.score(xtest,ytest)
precision =  metrics.precision_score(ytest, predict,  average='macro')
recall = metrics.recall_score(ytest, predict, average ='macro')
loss01 = metrics.zero_one_loss(ytest, predict)
timetrain = endTrain - startTrain
timePred = endpred - startpred

print("Ce modèle de KNN, K=3, à un score de ", score*100, "%.")
print("4eme image : prédiction ",predict[3], "reel : ", ytest[3])
print ("ce KNN à une précision de", precision*100, "%.")
print ("ce KNN à un recall de",recall*100, "%.")
print ("ce KNN à un zero-one_loss de",recall*100, "%.")
print("    temps apprentissage : ", timetrain, "sec , temps prediction = ", timePred, "sec." )

#Ce modèle de KNN, K=3, à un score de  95.75 %.
#4eme image : prédiction  2.0 reel :  2.0
#ce KNN à une précision de 95.8171550275235 %.
#ce KNN à un recall de 95.69581521050152 %.
#ce KNN à un zero-one_loss de 95.69581521050152 %.
#    temps apprentissage :  1.5378844738006592 sec , temps prediction =  29.139420747756958 sec.

In [ ]:
# MEILLEUR MLP 
# réseau 3 (50, 100, 200, 100,50)
# Relu et Adam (par défaut)
# petit alpha 0,1

#import necessaires
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split 
from sklearn.neural_network import MLPClassifier
from sklearn import metrics
import time

mnist=datasets.fetch_mldata('MNIST original')
#randomise Data et target
indices = np.random.randint(70000, size=70000)
data = mnist.data[indices]
target = mnist.target[indices]
# on veut un training set de 49000
xtrain, xtest, ytrain, ytest =train_test_split(data, target, train_size=49000)

clf3 = MLPClassifier(hidden_layer_sizes=(50, 100, 200, 100,50),alpha= 0.1) # defaut ADAM et Relu
startTrain =time.time()
clf3.fit(xtrain, ytrain)
endTrain = time.time()
# Predict
startpred= time.time()
predict = clf3.predict(xtest)
endpred = time.time()
score = clf3.score(xtest,ytest)

recall = metrics.recall_score(ytest, predict, average ='macro')
precision = metrics.precision_score(ytest, predict,  average='macro')
loss01 = metrics.zero_one_loss(ytest, predict)
timetrain = endTrain - startTrain
timePred = endpred - startpred


print("Ce modèle de MLP, de 5 couches de 50, 100, 200, 100, 50, a un score de ", score*100, "%.")
print("4eme image : prédiction ",predict[3], "reel : ", ytest[3])
print ("ce modèle de MLP à une précision de", precision*100, "%.")
print ("ce modèle de MLP à un recall de",recall*100, "%.")
print ("ce modèle de MLP à un zero-one_loss de",recall*100, "%.")
print("    temps apprentissage : ", timetrain, "sec , temps prediction = ", timePred, "sec." )

#Ce modèle de MLP, d' 5 couches de 50, 100, 200, 100, 50, a un score de  98.5047619047619 %.
#4eme image : prédiction  5.0 reel :  5.0
#ce modèle de MLP à une précision de 98.50503688809177 %.
#ce modèle de MLP à un recall de 98.49104431028593 %.
#ce modèle de MLP à un zero-one_loss de 98.49104431028593 %.
#    temps apprentissage :  322.9891335964203 sec , temps prediction =  0.6702065467834473 sec.

## Conclusion
<a id="conclusion"></a>
Ce rapport propose des paramétrage de 3 méthodes d'apprentissage supervisé pour classifier les données mnist. 
Les 3 méthodes portent leur fruits et présentes toutes des avantages et inconvénient.

Il serait utile de poursuivre l'étude en l'étendant à d'autres méthodes supervisées comme le décision tree, les réseaux baésian, les CNN et RNN.

